In [ ]:
import tensorflow as tf

strategy=tf.distribute.MirroredStrategy()
devices=strategy.num_replicas_in_sync

print("no of devices: {}".format(strategy.num_replicas_in_sync))

2023-04-28 08:48:45.396727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 08:48:45.642177: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
#at this point the dataset has been created and now we will serve it to the model
#but since the data can't be loaded directly into the memory
#we will use tensorflow dataset to efficiently load the data in batches

import tensorflow as tf
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback
import pandas as pd
import os
from tensorflow_addons.optimizers import AdamW ,LAMB
import cv2
import zipfile
import glob
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
import os
import tensorflow_addons as tfa
from focal_loss import SparseCategoricalFocalLoss


In [ ]:
class Image_classifier_on_zipped_data:
    def __init__(self,names,path_to_extract=None,zip_direc=None,
                 batch_per_gpu=16,
                image_shapes=(400,1400)):
        
        # constructor
        strategy=tf.distribute.MirroredStrategy()
        devices=strategy.num_replicas_in_sync
        self.batch_per_gpu=batch_per_gpu
        self.image_shapes=(400,1400)
        self.input_shape=(None,self.image_shapes[0],self.image_shapes[1],3)
        self.batch_size=self.batch_per_gpu*devices
        self.input_shape=(None,self.image_shapes[0],self.image_shapes[1],3)
        if(path_to_extract==None):
            self.path_to_extract=zip_direc #destination of output path
        if(zip_direc is not None):
            self.zip_direc=os.path.join(zip_direc,"**.zip")
        else:
            self.zip_direc=zip_direc
        self.names=names #order in which we want to pass the data
        
        
        
    def white_padding_and_scaling(self,default_shape,file_loc,overwrite=False):
        """
        2- adds white padding wherever necessary
        3- takes bitwise NOT transformation this esentially inverts the image sets black -0 as background while 
        255 is set as foreground
        4- if overwrite true then makes a new file with '_t' suffix 
        """
        try:
            img_array=cv2.imread(file_loc)
            shape=img_array.shape
        except:
            print("error in white padding--",file_loc)
            print("hence removing the file",file_loc)
            os.remove(file_loc)
            return
        
        padding_height=0
        padding_width=0
        crop_width=False
        crop_height=False

        if(shape[0]<=default_shape[0]): #if img is small in width then we need padding then 
            padding_height=default_shape[0]-shape[0]
        else:
            crop_height=True
            padding_height=0
        if(shape[1]<=default_shape[1]):
            padding_width=default_shape[1]-shape[1]
        else:
            crop_width=True
            padding_width=0
        if(padding_width>0 or padding_height>0):
            colour_fill=(255,255,255) #colour to pad this is white
            new_array=cv2.copyMakeBorder(img_array, 0,padding_height , 0, padding_width, cv2.BORDER_CONSTANT,value=colour_fill)
        else:
            new_array=img_array[0:default_shape[0], 0:default_shape[1]]

        if(crop_width==True):
            new_array=new_array[0:default_shape[0], 0:default_shape[1]]
        if(crop_height==True):
            new_array=new_array[0:default_shape[0], 0:default_shape[1]]


        new_array=cv2.bitwise_not(new_array)
        if(overwrite==True):
            new_name=file_loc.replace(".png","_t.png")
            #print(new_name)
            cv2.imwrite(new_name,new_array)
            os.remove(file_loc)
            return

        return new_array
    
    def append_last_run(self,val,log_file):
        with open(log_file,"a") as f:
            f.write(val)
            
    def get_last_run(self):
        with open(self.log_file,"r") as f:
            lines=f.readlines()
        f.close()

        files=[]
        for line in lines:
            files.append(line.split(",")[0])

        return files
    
    def calc_class_weigths(self,_dir):
        sub_dir=list(filter(lambda x: x.startswith("label") ,os.listdir(_dir)))
        sub_dir.sort()
        class_names=[]
        for folder in sub_dir:
            glob_path=os.path.join(_dir,folder)+"/**_t.png"
            files=glob.glob(glob_path)
            class_names.append(len(files))

        class_weights=np.array(class_names)
        weights=np.full(shape=len(class_weights),fill_value=sum(class_weights))
        out = np.divide(weights, class_weights)

        class_weights=np.around(out)
        class_weight_dict={}
        for i,weight in enumerate(class_weights):
            class_weight_dict[i]=weight


        return class_weight_dict
    
    def fit(self,model,model_path=None,validation_data=None,n_jobs=-2,class_weight=None,log_file="my_model.txt"):
        
        order=[]
        names=self.names
        
        if(self.zip_direc is not None):
            print(zip_direc)
            zip_files=glob.glob(self.zip_direc)
            self.log_file=log_file

            #building the right order of execution

            for zip_file in zip_files:
                chk_val=zip_file[:-4].split("/")[-1]
                if(chk_val in names):
                    order.append(zip_file)
        

        for name in names:
            full_name=os.path.join(os.getcwd(),name)
            if(full_name+".zip" not in order):
                order.append(full_name)
                
                
        i=1
        for zip_file in order: #take a batched zip file unzip it in $SCRATCH dir
            self.log_file=log_file
            
            try:
                run_so_far=self.get_last_run()
                if(not zip_file.endswith(".zip")):
                    zip_file+=".zip"
                    
                if(zip_file[:-4] in run_so_far): #or file in skips
                    print("already ran on:",zip_file[:-4])
                    continue
                else:
                    print("not in previous runs, hence running on this batch")
                    print(zip_file)
            except FileNotFoundError:
                print("check for ",log_file)
                pass           
            except:
                pass
            
            print("--unzipping file--")
            
            #skip if unzipped folder is there
            skip_dir=False
            if(os.path.isdir(zip_file)):
                zip_file=zip_file+".zip"
                skip_dir=True
                
            if(not os.path.exists(zip_file[:-4]) and skip_dir==False):
                with zipfile.ZipFile(zip_file, "r") as zip_ref:
                    print(f"extracting {zip_file} at--",self.path_to_extract)
                    zip_ref.extractall(self.path_to_extract)

            
            #we need to transform these images 
            path_chk=zip_file[:-4]+"/**/**.png"
            png_files=glob.glob(path_chk)
            
            
            filtered_files=list(filter(lambda x: not x.endswith("_t.png"),png_files))
            
            bad_files=list(filter(lambda x:  x.endswith("_t_t.png"),png_files))
            if(len(bad_files)!=0):
                print("many reruns please check the data again")
                return 
            
            
            
            print("--running transformations")
            print(f"len of files to run in{zip_file[:-4]} :{len(filtered_files)}")
            

            res=Parallel(n_jobs=n_jobs,backend="threading",verbose=2)(delayed(self.white_padding_and_scaling)
                                                       (default_shape=image_shapes,file_loc=fname,overwrite=True) for fname in tqdm(filtered_files))
            
            #converting it in the tf dataset object
            print("--converting it in tf dataset object")
            if(class_weight=="balanced"):
                class_weight_dict=self.calc_class_weigths(zip_file[:-4])
                
            
            train_dataset_batch=tf.keras.preprocessing.image_dataset_from_directory(directory=zip_file[:-4],
                                                                                    image_size=image_shapes,
                                                                                    batch_size=batch_size,
                                                                                    seed=2,
                                                                                    shuffle=True)
            
            train_dataset_batch=train_dataset_batch.prefetch(buffer_size=tf.data.AUTOTUNE)
            if(validation_data!=None):
                validation_data=validation_data.prefetch(buffer_size=tf.data.AUTOTUNE)
            with strategy.scope():
                if(model_path is not None and os.path.exists(model_path)):
                    print(f"model exists in{model_path}")
                    ##change this <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< uncomment
                    model = tf.keras.models.load_model(model_path)
                    print("="*20)
                    print(model.summary())
                    print("="*20)
                else:
                    model_path=log_file.replace(".txt",".h5")
                    
                
                if(class_weight is None):
                    history = model.fit(train_dataset_batch, epochs=1,validation_data=validation_data)
                    pass
                elif(class_weight=="balanced"):
                    print(f"class_weight is balanced{class_weight_dict}")
                    history = model.fit(train_dataset_batch,class_weight=class_weight_dict,epochs=1)
                    #val_loss,val_acc=model.evaluate(validation_data)
                
                
                #metrics after a batch
                train_loss=history.history['loss']
                train_acc=history.history['accuracy']
                
                if(validation_data is not None):
                    val_loss=history.history['val_loss']
                    val_acc=history.history['val_accuracy']
                else:
                    val_loss="NA"
                    val_acc="NA"
                

                line_to_write=f'{zip_file[:-4]},{train_loss},{val_loss},{train_acc},{val_acc}\n'
                self.append_last_run(line_to_write,log_file)
                
                #saving the model
                save_model=model_path[:-3]+str(i)+".h5"
                #print(save_model)
                model.save(save_model)
                i+=1

            

In [ ]:
######loading small validation data incase we are not abe to upload full validation data
image_shapes=(400,1400)
batch_per_gpu=4 #####<<<<<<<<<<<<<<<<<<<<<<<<<<<change this to fit in gpu
batch_size=batch_per_gpu*devices

sub_sample_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory="/gpfsscratch/rech/zpf/uyf36me/validation_patches",
    image_size=image_shapes,
    batch_size=batch_size,
    seed=2,
    shuffle=True
    )

sub_sample_validation_dataset=sub_sample_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

######loading small validation data incase we are not abe to upload full validation data
image_shapes=(400,1400)
batch_per_gpu=4 #####<<<<<<<<<<<<<<<<<<<<<<<<<<<change this to fit in gpu
batch_size=batch_per_gpu*devices

full_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory="/gpfsscratch/rech/zpf/uyf36me/vision_patches/val_data",
    image_size=image_shapes,
    batch_size=batch_size,
    seed=2,
    shuffle=True
    )

full_validation_dataset=full_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#reordering file name accroding to some predefined run (which is here nlp)
train_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/**.csv"
import glob
files=glob.glob(train_path)
file_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/start_mlm_forft_res.txt"
import os
base="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/"
with open(file_path,"r") as f:
    lines=f.readlines()
start=[]
for line in lines:
    val=line.split(",")[0][2:]
    start.append(val[:-4].rsplit("/")[1])

#get the order of execution of batch        
order=start

In [ ]:
order

In [7]:
pngs=glob.glob("/gpfsscratch/rech/zpf/uyf36me/training_patches/batch_2/**/**.png")
print(len(pngs))
filtered_files=list(filter(lambda x: not x.endswith("_t.png"),pngs))
len(filtered_files)

187398


0

In [ ]:
rescaling_layer=tf.keras.layers.Rescaling(scale=1./255) #we will use this later

class efficientnetv2m:  #<<<<<<<<<<<<<<change this
    @staticmethod
    def build(input_shape, classes,weights=None,pooling=None):
        #<<<<<<<<<<<<<<change this
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        #<<<<<<<<<<<<<<change this
        base_model=tf.keras.applications.efficientnet_v2.EfficientNetV2M(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling="avg") #<<<<<
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        if(pooling==None):
            model.add(tf.keras.layers.Flatten())
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model
    


image_shapes=(400,1400)
input_shape=(None,image_shapes[0],image_shapes[1],3)
NUM_EPOCHS=1
learning_rate=2e-5
classes=4
zip_direc="/gpfsscratch/rech/zpf/uyf36me/vision_patches"
opt=LAMB(learning_rate)
#opt=AdamW(learning_rate) #for focal loss only
#opt=tf.keras.optimizers.Adam(learning_rate) #trying for foc loss

with strategy.scope():
    #<<<<<<<<<<<<<<<<<<<<<<change this
    model = efficientnetv2m.build(input_shape=input_shape,
                               classes=classes) 


    #loss is sparse because we have int features
    loss_focal=tfa.losses.SigmoidFocalCrossEntropy() #only for binary classification
    
    default_loss="sparse_categorical_crossentropy"
    #code https://github.com/artemmavrin/focal-loss 
    #paper Focal Loss for Dense Object Detection
    #loss_focal=SparseCategoricalFocalLoss(gamma=2)
    model.compile(loss=default_loss, optimizer=opt, metrics=["accuracy"])
    model.build(input_shape=input_shape)

    print(model.summary())


layers_depth_len=len(model.layers)
total_params=(model.count_params())
config=dict(num_epochs=NUM_EPOCHS,
           model_name="efficientnetb7",
           total_params=total_params,
           layers_depth_len=layers_depth_len)



#opt=tf.keras.optimizers.Adam()

#if any batch not present then it will be skipped
location_batches="/gpfsscratch/rech/zpf/uyf36me/training_patches"
batches=os.listdir(location_batches)
names=[os.path.join(location_batches,batch) for batch in batches if batch.startswith("batch")]
#we need to reorder batches now
order=['batch_28', 'batch_18', 'batch_7', 'batch_2', 'batch_0', 'batch_108', 'batch_106', 'batch_85',
       'batch_123', 'batch_48', 'batch_114', 'batch_125', 'batch_67', 'batch_4', 'batch_93', 'batch_3',
       'batch_86', 'batch_36', 'batch_41']


new_names=[]
for batch in order:
    for name in names:
        if(name.endswith(batch)):
            new_names.append(name)
            

names=new_names

clf=Image_classifier_on_zipped_data(names,batch_per_gpu=batch_per_gpu)

#, validation_data=sub_sample_validation_dataset

#provide model or model_path
#<<<<<<<<<<<<<<<<<<<<<<change this
files=clf.fit(model=model,model_path="./new_models/r_efficientnetv2m_avg9.h5",
        log_file="./new_models/r_efficientnetv2m_avg.txt",class_weight=None) #validation_data=sub_sample_validation_dataset

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

  0%|          | 0/40153 [00:00<?, ?it/s][Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done  19 tasks      | elapsed:    0.1s
100%|██████████| 40153/40153 [01:28<00:00, 454.14it/s]
[Parallel(n_jobs=-2)]: Done 40153 out of 40153 | elapsed:  1.5min finished


--converting it in tf dataset object
Found 41470 files belonging to 4 classes.
model exists in./new_models/r_efficientnetv2m_avg9.h5
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 400, 1400, 3)      0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 flatten_2 (Flatten)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 53,155,512
Trainable params: 52,863,480
Non-trainable params: 292,032
___________________________________

2023-04-28 08:45:38.568277: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 41470
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:8"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}



INFO:tensorflow:batch_all_reduce: 649 all-reduces with algorithm = nccl, num_packs = 1


In [ ]:
#70.2 app for focal loss ep 1 lamb optmizer

#234/2680 [=>............................] - ETA: 40:05 - loss: 1.2880 - accuracy: 0.5186 pool avg #20 mill
#291/2680 [==>...........................] - ETA: 38:28 - loss: 0.8468 - accuracy: 0.6633 no pool  #23 mill
#235/2680 [=>............................] - ETA: 39:27 - loss: 4.8293 - accuracy: 0.3949 pool max #20 mill

#hence we choose no pooling

#around 20000 steps of 16 bsz on 4 a100 gives 75% accuracy
 default_loss

In [ ]:
#23,260,004 726-0.84 0.66

In [ ]:
# from keras_flops import get_flops
# flops = get_flops(model, batch_size=1)

# print("="*20)
# print(f"FLOPS: {flops / 10 ** 9:.03} G")
# print("="*20)

In [ ]:
import glob
demo=glob.glob("/gpfsscratch/rech/zpf/uyf36me/vision_patches/batch_28/**/**.png")
from PIL import Image
from IPython.display import Image as Img

img = Image.open(demo[0])
img

In [ ]:
batch_per_gpu=8
image_shapes=(400,1400)
batch_size=batch_per_gpu*devices
input_shape=(None,image_shapes[0],image_shapes[1],3)
NUM_EPOCHS=1
learning_rate=2e-5
classes=4
zip_direc="/gpfsscratch/rech/zpf/uyf36me/vision_patches"
opt=LAMB(learning_rate)

#if any batch not present then it will be skipped
names=['batch_28.zip','batch_18.zip','batch_108.zip',
     'batch_106.zip','batch_85.zip','batch_123.zip',
     'batch_7.zip','batch_48.zip','batch_2.zip','batch_114.zip',
     'batch_125.zip','batch_67.zip','batch_4.zip']

rescaling_layer=tf.keras.layers.Rescaling(scale=1./255) #we will use this later

class resnet50: 
    @staticmethod
    def build(input_shape, classes,weights=None,pooling=None):
        
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.resnet50.ResNet50(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,pooling=pooling)
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        if(pooling==None):
            model.add(tf.keras.layers.Flatten())
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model
    
#loading small validation data incase we are not abe to upload full validation data

# sub_sample_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
#     directory="/gpfsscratch/rech/zpf/uyf36me/vision_patches/sub_sample_data",
#     image_size=image_shapes,
#     batch_size=batch_size,
#     seed=2,
#     shuffle=True
#     )

# sub_sample_validation_dataset=sub_sample_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
# with strategy.scope():
#     model = resnet50.build(input_shape=input_shape,
#                                classes=classes) 


#     #loss is sparse because we have int features
#     model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#     model.build(input_shape=input_shape)

#     print(model.summary())


# from keras_flops import get_flops
# flops = get_flops(model, batch_size=1)

# print("="*20)
# print(f"FLOPS: {flops / 10 ** 9:.03} G")
# print("="*20)


layers_depth_len=len(model.layers)
total_params=(model.count_params())
config=dict(num_epochs=NUM_EPOCHS,
           model_name="Resnet50-Imagenet",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

#print(config)
    
clf=Image_classifier_on_zipped_data(zip_direc,names)
#clf.fit(model,log_file="Resnet.txt",validation_data=None)

In [ ]:
rescaling_layer=tf.keras.layers.Rescaling(scale=1./255) #we will use this later

class densenet169: 
    @staticmethod
    def build(input_shape, classes,weights=None,pooling=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.densenet.DenseNet169(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,pooling=pooling)
        
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        if(pooling==None):
            model.add(tf.keras.layers.Flatten())
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model
    
#loading small validation data incase we are not abe to upload full validation data

# sub_sample_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
#     directory="/gpfsscratch/rech/zpf/uyf36me/vision_patches/sub_sample_data",
#     image_size=image_shapes,
#     batch_size=batch_size,
#     seed=2,
#     shuffle=True
#     )

# sub_sample_validation_dataset=sub_sample_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
with strategy.scope():
    model = densenet169.build(input_shape=input_shape,
                               classes=classes) 


    #loss is sparse because we have int features
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.build(input_shape=input_shape)

    print(model.summary())





layers_depth_len=len(model.layers)
total_params=(model.count_params())
config=dict(num_epochs=NUM_EPOCHS,
           model_name="densenet169-Imagenet",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

#print(config)
    
clf=Image_classifier_on_zipped_data(zip_direc,names)

In [ ]:
rescaling_layer=tf.keras.layers.Rescaling(scale=1./255) #we will use this later

class efficientnetB0: 
    @staticmethod
    def build(input_shape, classes,weights=None,pooling=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.efficientnet.EfficientNetB0(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling=pooling)
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        if(pooling==None):
            model.add(tf.keras.layers.Flatten())
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model
    
#loading small validation data incase we are not abe to upload full validation data

# sub_sample_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
#     directory="/gpfsscratch/rech/zpf/uyf36me/vision_patches/sub_sample_data",
#     image_size=image_shapes,
#     batch_size=batch_size,
#     seed=2,
#     shuffle=True
#     )

# sub_sample_validation_dataset=sub_sample_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
with strategy.scope():
    model = efficientnetB0.build(input_shape=input_shape,
                               classes=classes) 


    #loss is sparse because we have int features
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.build(input_shape=input_shape)

    print(model.summary())





layers_depth_len=len(model.layers)
total_params=(model.count_params())
config=dict(num_epochs=NUM_EPOCHS,
           model_name="densenet169-Imagenet",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

#print(config)
    
clf=Image_classifier_on_zipped_data(zip_direc,names)

In [ ]:
#8 batch per gpu


In [ ]:
rescaling_layer=tf.keras.layers.Rescaling(scale=1./255) #we will use this later

class Convnext: 
    @staticmethod
    def build(input_shape, classes,weights=None,pooling=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.convnext.ConvNeXtSmall(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling=pooling)
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        if(pooling==None):
            model.add(tf.keras.layers.Flatten())
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model
    
#loading small validation data incase we are not abe to upload full validation data

# sub_sample_validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(
#     directory="/gpfsscratch/rech/zpf/uyf36me/vision_patches/sub_sample_data",
#     image_size=image_shapes,
#     batch_size=batch_size,
#     seed=2,
#     shuffle=True
#     )

# sub_sample_validation_dataset=sub_sample_validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
with strategy.scope():
    model = Convnext.build(input_shape=input_shape,
                               classes=classes) 


    #loss is sparse because we have int features
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.build(input_shape=input_shape)

    print(model.summary())





layers_depth_len=len(model.layers)
total_params=(model.count_params())
config=dict(num_epochs=NUM_EPOCHS,
           model_name="densenet169-Imagenet",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

#print(config)
    
clf=Image_classifier_on_zipped_data(zip_direc,names)
#clf.fit(model,log_file="Resnet.txt",validation_data=None)

# Avoiding the gradient vanishing problem with skip connections (ResNets)

Resnets (2015): https://arxiv.org/pdf/1512.03385.pdf

**Proposed 3 variants:**

1. ResNet-50
2. ResNet-101
3. ResNet-152

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://media.geeksforgeeks.org/wp-content/uploads/20200424011138/ResNet.PNG")

In [ ]:
class resnet50: 
    @staticmethod
    def build(input_shape, classes,weights=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.resnet50.ResNet50(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling='avg')
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model

#adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#ideally we should double the epochs when drop out is 0.5
NUM_EPOCHS = 30 

model = resnet50.build(input_shape=input_shape,
                           classes=classes) 


#loss is sparse because we have int features
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.build(input_shape=input_shape)

print(model.summary())


layers_depth_len=len(model.layers)
total_params=(model.count_params())

config=dict(num_epochs=NUM_EPOCHS,
           model_name="Resnet50-Imagenet",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

print(config)


wandb.login(key="d3c2d30bb5cdd697f78c2ab583f9bde909c7c7a8")
run=wandb.init(project="vision_approaches",
               config=config
              )

config=wandb.config


#early stopping to see the training 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', min_delta=0.01,patience = 3)

##checkpoint callback
checkpoint_filepath="./checkpoint/"+config["model_name"]+"/"+"my_model.h5"

print(checkpoint_filepath)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

### tensorboard callback
tensorboard_cb=tf.keras.callbacks.TensorBoard(log_dir="my_log_dir",histogram_freq=1)

#wandb callback
wb_cb=WandbCallback()

print("Training network...")

#approimately 200h per epoch
history = model.fit(train_dataset, validation_data=validation_dataset,epochs=NUM_EPOCHS,verbose=1,callbacks=[early_stopping_cb,model_checkpoint_cb,wb_cb])

In [ ]:
run.finish()

In [ ]:
#one thing we can notice here is that the FLOPS and parameters of models have significantly increased
#2016-2018 the focus was to build even deeper networks with least amount of FLOPS while also increasing the accuracy

In [ ]:
#Densenets 2017
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/1400/1*rmHdoPjGUjRek6ozH7altw.png")

#everything is connected to everything from it's past 

In [ ]:
Image(url= "https://miro.medium.com/max/1400/1*k75kOqrISzAEgtS15WWSZA.png")

In [ ]:
# Densenet variants variants 121/169/201

class densenet169: 
    @staticmethod
    def build(input_shape, classes,weights=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.densenet.DenseNet169(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling='avg')
        #print(base_model.summary())
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model

#adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#ideally we should double the epochs when drop out is 0.5
NUM_EPOCHS = 30 

model = densenet169.build(input_shape=input_shape,
                           classes=classes) 


#loss is sparse because we have int features
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.build(input_shape=input_shape)

print(model.summary())


layers_depth_len=len(model.layers)
total_params=(model.count_params())

config=dict(num_epochs=NUM_EPOCHS,
           model_name="densenet169",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

print(config)


wandb.login(key="d3c2d30bb5cdd697f78c2ab583f9bde909c7c7a8")
run=wandb.init(project="vision_approaches",
               config=config
              )

config=wandb.config


#early stopping to see the training 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', min_delta=0.01,patience = 3)

##checkpoint callback
checkpoint_filepath="./checkpoint/"+config["model_name"]+"/"+"my_model.h5"

print(checkpoint_filepath)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

### tensorboard callback
tensorboard_cb=tf.keras.callbacks.TensorBoard(log_dir="my_log_dir",histogram_freq=1)

#wandb callback
wb_cb=WandbCallback()

print("Training network...")

#approimately 500h per epoch
history = model.fit(train_dataset, validation_data=validation_dataset,epochs=NUM_EPOCHS,verbose=1,callbacks=[early_stopping_cb,model_checkpoint_cb,wb_cb])

In [ ]:
run.finish()

In [ ]:
#NASnet neural arch search to find the right architecture on Cifar data that also works on Imagenet
#the idea is to get a general machine searched architecture
#policy gradient 
#28% lower FLOPS means faster to train
#tensorflow comes with 2 variants Mobile/Large
#we can try the large model should not be too bad in terms of computation

# Densenet variants variants 121/169/201

class Nasnet: 
    @staticmethod
    def build(input_shape, classes,weights=None):
        new_input_shape=(input_shape[1],input_shape[2],input_shape[3])
        base_model=tf.keras.applications.nasnet.NASNetLarge(
                                        include_top=False,
                                        weights=weights,
                                        input_shape=new_input_shape,
                                        pooling='avg')
        
        print(len(base_model.layers))
        model = tf.keras.Sequential()
        model.add(rescaling_layer)
        model.add(base_model)
        # softmax classifier
        model.add(tf.keras.layers.Dense(classes, activation='softmax'))
        return model

#adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#ideally we should double the epochs when drop out is 0.5
NUM_EPOCHS = 30 

model = Nasnet.build(input_shape=input_shape,
                           classes=classes) 


#loss is sparse because we have int features
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.build(input_shape=input_shape)

print(model.summary())


layers_depth_len=len(model.layers)
total_params=(model.count_params())

config=dict(num_epochs=NUM_EPOCHS,
           model_name="NasnetLarge",
           total_params=total_params,
           layers_depth_len=layers_depth_len)

print(config)


wandb.login(key="d3c2d30bb5cdd697f78c2ab583f9bde909c7c7a8")
run=wandb.init(project="vision_approaches",
               config=config
              )

config=wandb.config


#early stopping to see the training 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor ='val_loss', min_delta=0.01,patience = 3)

##checkpoint callback
checkpoint_filepath="./checkpoint/"+config["model_name"]+"/"+"my_model.h5"

print(checkpoint_filepath)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

### tensorboard callback
tensorboard_cb=tf.keras.callbacks.TensorBoard(log_dir="my_log_dir",histogram_freq=1)

#wandb callback
wb_cb=WandbCallback()

print("Training network...")

#approimately  
history = model.fit(train_dataset, validation_data=validation_dataset,epochs=NUM_EPOCHS,verbose=1,callbacks=[early_stopping_cb,model_checkpoint_cb,wb_cb])